In [1]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
from IPython.display import HTML
from IPython.display import display

# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
code_show=false; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.''')
display(tag)
%cd /content/drive/MyDrive/RohithWorkspace/Object-Detection-Website
!git config --global user.email "007rohitjj04@gmail.com"
!git config --global user.name "Rohith04MVK"
!git add -A
!git commit -m "Still testing google colab"
!git push